In [2]:
import requests
import pandas as pd
from datetime import timedelta
from datetime import datetime

from airflow import DAG
from airflow.operators.python import PythonOperator

TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'


def get_data():
    # Здесь пока оставили запись в файл, как передавать переменую между тасками будет в третьем уроке
    top_doms = pd.read_csv(TOP_1M_DOMAINS)
    top_data = top_doms.to_csv(index=False)

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)

        

def top_10_domain_zones_dgati():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_df['domain_zone'] = top_data_df['domain'].str.split('.').apply(lambda x: x[-1])
    top_10_dz =  top_data_df.groupby('domain_zone', as_index=False) \
                        .agg({'domain': 'count'}) \
                        .rename(columns={'domain': 'count_of_names'}) \
                        .sort_values('count_of_names', ascending=False) \
                        .head(10)
    with open('top_10_domain_zones_dgati.csv', 'w') as f:
        f.write(top_10_dz.to_csv(index=False, header=False))
        
def get_longest_name_dgati():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_df['domain_length'] = top_data_df['domain'].apply(lambda x: len(x))
    longest_name =  top_data_df[top_data_df.domain_length == top_data_df.domain_length.max()] \
                                                                .sort_values('domain') \
                                                                .head(1) \
                                                                .domain
    with open('longest_name_dgati.csv', 'w') as f:
        f.write(longest_name.to_csv(index=False, header=False))

def where_is_airflow_dgati():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    temp_df = top_data_df[top_data_df['domain'].apply(lambda x: True if 'airflow.com' in x else False)]
    if len(temp_df) > 0:
        where_is_airflow = top_data_df[top_data_df['domain'].apply(lambda x: True if 'airflow.com' in x else False)] \
                                .reset_index()['rank']
    else:
        where_is_airflow = pd.Series('Нет такого домена')
        
    with open('where_is_airflow.csv', 'w') as f:
        f.write(where_is_airflow.to_csv(index=False, header=False))
    
    
    

# def get_stat():
#     top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
#     top_data_top_10 = top_data_df[top_data_df['domain'].str.endswith('.ru')]
#     top_data_top_10 = top_data_top_10.head(10)
#     with open('top_data_top_10.csv', 'w') as f:
#         f.write(top_data_top_10.to_csv(index=False, header=False))


# def get_stat_com():
#     top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
#     top_data_top_10 = top_data_df[top_data_df['domain'].str.endswith('.com')]
#     top_data_top_10 = top_data_top_10.head(10)
#     with open('top_data_top_10_com.csv', 'w') as f:
#         f.write(top_data_top_10.to_csv(index=False, header=False))


def print_data(ds):
#     with open('top_data_top_10.csv', 'r') as f:
#         all_data = f.read()
#     with open('top_data_top_10_com.csv', 'r') as f:
#         all_data_com = f.read()
    
    with open('top_10_domain_zones_dgati.csv', 'r') as f:
        top_10_domain_zones_dgati = f.read()
    with open('longest_name_dgati.csv', 'r') as f:
        longest_name_dgati = f.read()
    with open('where_is_airflow.csv', 'r') as f:
        where_is_airflow = f.read() 
    
    date = ds

    print(f'Top 10 domains zones for date {date}')
    print(top_10_domain_zones_dgati)

    print(f'Domain with longest name for date {date}')
    print(longest_name_dgati)
    
    print(f'Rank of airflow.com for date {date} is')
    print(where_is_airflow)


default_args = {
    'owner': 'd.gatiatullin',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 11, 20),
}
schedule_interval = '0 17 * * *'

dag = DAG('dgati_dag11', default_args=default_args, schedule_interval=schedule_interval)

t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)

t2 = PythonOperator(task_id='top_10_domain_zones_dgati',
                    python_callable=top_10_domain_zones_dgati,
                    dag=dag)

t3 = PythonOperator(task_id='get_longest_name_dgati',
                        python_callable=get_longest_name_dgati,
                        dag=dag)

t4 = PythonOperator(task_id='where_is_airflow_dgati',
                        python_callable=where_is_airflow_dgati,
                        dag=dag)

t5 = PythonOperator(task_id='print_data',
                    python_callable=print_data,
                    dag=dag)

t1 >> [t2, t3, t4] >> t5

<Task(PythonOperator): print_data>

In [7]:
top_data_df = pd.read_csv(TOP_1M_DOMAINS, names=['rank', 'domain'])

In [8]:
top_data_df['domain_zone'] = top_data_df['domain'].str.split('.').apply(lambda x: x[-1])

In [10]:
top_data_df.groupby('domain_zone', as_index=False) \
                        .agg({'domain': 'count'}) \
                        .rename(columns={'domain': 'count_of_names'}) \
                        .sort_values('count_of_names', ascending=False) \
                        .head(10)[['domain_zone']]

,domain_zone
146,com
474,org
449,net
539,ru
173,de
459,nl
34,au
333,it
650,uk
496,pl


In [5]:
top_data_df

,rank,domain
0,1,google.com
1,2,youtube.com
2,3,baidu.com
3,4,facebook.com
4,5,qq.com
...,...,...
648357,648358,xsswary.com
648358,648359,yasaloonak.net
648359,648360,ymhstore.co.il
648360,648361,yobit.biz


In [6]:
top_data_df['domain_zone'] = top_data_df['domain'].str.split('.').apply(lambda x: x[-1])

In [7]:
top_data_df

,rank,domain,domain_zone
0,1,google.com,com
1,2,youtube.com,com
2,3,baidu.com,com
3,4,facebook.com,com
4,5,qq.com,com
...,...,...,...
648357,648358,xsswary.com,com
648358,648359,yasaloonak.net,net
648359,648360,ymhstore.co.il,il
648360,648361,yobit.biz,biz


In [8]:
top_10_dz =  top_data_df.groupby('domain_zone', as_index=False) \
                        .agg({'domain': 'count'}) \
                        .rename(columns={'domain': 'count_of_names'}) \
                        .sort_values('count_of_names', ascending=False) \
                        .head(10)

In [9]:
top_10_dz

,domain_zone,count_of_names
146,com,322495
474,org,26597
449,net,24778
539,ru,17448
173,de,11401
459,nl,9850
34,au,9512
333,it,9092
650,uk,8766
496,pl,7955


In [10]:
with open('top_10_domain_zones_dgati.csv', 'w') as f:
        f.write(top_10_dz.to_csv(index=False, header=False))

In [ ]:

    top_10_dz =  top_data_df.groupby('domain_zone', as_index=False) \
                        .agg({'domain': 'count'}) \
                        .rename(columns={'domain': 'count_of_names'}) \
                        .sort_values('count_of_names', ascending=False) \
                        .head(10)
    with open('top_10_domain_zones_dgati.csv', 'w') as f:
        f.write(top_10_dz.to_csv(index=False, header=False))

In [3]:
top_data_df = pd.read_csv(TOP_1M_DOMAINS, names=['rank', 'domain'])

In [4]:
top_data_df

,rank,domain
0,1,google.com
1,2,youtube.com
2,3,baidu.com
3,4,bilibili.com
4,5,facebook.com
...,...,...
560172,560173,zadnit.net
560173,560174,zahra.com.sa
560174,560175,zerointegration.com
560175,560176,zerops.io


In [7]:
top_data_df[top_data_df['domain'].apply(lambda x: True if 'airflow.com' in x else False)]

,rank,domain
238231,238232,airflow.com
